# Imports

In [1]:
!pip install -q optuna
!pip install -q shap

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import optuna
from optuna.samplers import TPESampler
import warnings
import shap
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_csv('train.csv', index_col='id')

original = pd.read_csv('final_depression_dataset_1.csv', index_col='id')

original['Depression'] = original['Depression'].map({
    'No': 0,
    'Yes': 1
})

train = pd.concat(objs=[train, original])
train.head()

ValueError: Index id invalid

In [ ]:
test = pd.read_csv('test.csv', index_col='id')
test.head()

In [ ]:
train = train.fillna('None').astype('string')
test = test.fillna('None').astype('string')

y = train['Depression']
X = train.drop(['Depression'], axis=1)

In [ ]:
catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': 0.08114394459649094,
    'iterations': 1000,
    'depth': 6,
    'random_strength':0,
    'l2_leaf_reg': 0.7047064221215757,
    'task_type':'GPU',
    'random_seed':42,
    'verbose':False    
}

In [ ]:
cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(X, y)
scores = []
test_preds = []
X_test_pool = Pool(test, cat_features=X.columns.values)
for i, (train_idx, val_idx) in enumerate(cv_splits):
    model = CatBoostClassifier(**catboost_params)
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X.columns.values)
    X_valid_pool = Pool(X_val_fold, y_val_fold, cat_features=X.columns.values)
    model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=False, early_stopping_rounds=200)
    val_pred = model.predict(X_valid_pool)
    score = accuracy_score(y_val_fold, val_pred)
    scores.append(score)
    test_pred = model.predict_proba(X_test_pool)[:, 1]
    test_preds.append(test_pred)
    print(f'Fold {i + 1} accuracy_score: {score}')
print(f'Cross-validated accuracy_score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
print(f'Max accuracy_score score: {np.max(scores):.3f}')
print(f'Min accuracy_score score: {np.min(scores):.3f}')

# Submission

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e11/sample_submission.csv')
sample_submission['Depression'] = np.round(np.mean(test_preds, axis=0))
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [4]:
v3 = pd.read_csv('v3.csv')

In [5]:
v3

,id,Depression
0,140700,0.0
1,140701,0.0
2,140702,0.0
3,140703,1.0
4,140704,0.0
...,...,...
93795,234495,0.0
93796,234496,1.0
93797,234497,0.0
93798,234498,1.0


In [7]:
v3['Depression'].value_counts()

Depression
0.0    77403
1.0    16397
Name: count, dtype: int64